# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [2]:
from ipywidgets import *

In [3]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test


## Define dictionary

In [26]:
## Read ref_launch.yaml
# main_code_dir=config_dict[facility]['main_code_dir']
# dict_pars=config_dict[facility][img_size][run_type]
dict_pars={'nodes':1, 'queue':'regular','bins':'unenven','cores':64,'time':'0:30:00',
           'job_name':'3ptfnc',
            'suffix':'run118-18560_3d_64cube-32slice','num_l':8,'img_slice':32,
                 }

dict_pars['staging_loc']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/'
dict_pars['code_dir']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test'


In [27]:
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'

# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-124_step-19390.npy'

dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-118_step-18560.npy'



In [28]:
# %%bash -s "$op_fldr"
# ls $1

In [29]:
dict_pars

{'nodes': 1,
 'queue': 'regular',
 'bins': 'unenven',
 'cores': 64,
 'time': '0:30:00',
 'job_name': '3ptfnc',
 'suffix': 'run118-18560_3d_64cube-32slice',
 'num_l': 8,
 'img_slice': 32,
 'staging_loc': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/',
 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test',
 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-118_step-18560.npy'}

In [30]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir={code_dir}
fname={fname}

srun -n 64 python $code_dir/3pt_fcn_simple.py -n {num_l} --img_slice {img_slice} -idx $I -f $fname -sfx {suffix}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [31]:
fname='batch_3ptcn.sh'
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test/batch_3ptcn.sh


In [32]:
## Move to staging locations in project space:
os.chdir(dict_pars['staging_loc'])

In [33]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=0:30:00
#SBATCH --job-name=3ptfnc

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test
fname=/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-118_step-18560.npy

srun -n 64 python $code_dir/3pt_fcn_simple.py -n 8 --img_slice 32 -idx $I -f $fname -sfx run118-18560_3d_64cube-32slice

echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [34]:
lst=np.arange(32)
# lst=[0,3,6,8,19]
lst2=[str(i) for i in lst][5:]
strg=' '.join(lst2)
print(strg)

5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31


In [35]:
%%bash -s "$filename" "$strg" ## Use python variable in bash
# for i in $2; do echo $i; echo $1; done
for i in $2; do sbatch $1 $i; done

Submitted batch job 43059119
Submitted batch job 43059121
Submitted batch job 43059122
Submitted batch job 43059123
Submitted batch job 43059125
Submitted batch job 43059126
Submitted batch job 43059127
Submitted batch job 43059128
Submitted batch job 43059129
Submitted batch job 43059131
Submitted batch job 43059132
Submitted batch job 43059133
Submitted batch job 43059134
Submitted batch job 43059135
Submitted batch job 43059137
Submitted batch job 43059138
Submitted batch job 43059139
Submitted batch job 43059140
Submitted batch job 43059142
Submitted batch job 43059143
Submitted batch job 43059144
Submitted batch job 43059145
Submitted batch job 43059146
Submitted batch job 43059147
Submitted batch job 43059148
Submitted batch job 43059150
Submitted batch job 43059151


In [54]:
pwd

'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/simple_test'